# Let's have a picnic

* The Hatfields, Montagues, McCoys and Capulets are going on their annual family picnic.  
* Four cars are available to transport the families to the picnic.  
* The cars can carry the following numbers of people: car 1, 4; car 2, 3; car 3, 3; car 4, 4.
* There are four people in each family, and no car can carry more than two people from any one family. 
* Determine the maximum number of people that can be transported to the picnic.

In [ ]:
# Make the network 
#  Here I use julia Sets and union operator
s = Set([Symbol("source")])
t = Set([Symbol("sink")])
families = Set([:Hatfield, :Montague, :McCoy, :Capulet])
cars = Set(Symbol("car" * string(i)) for i in 1:4)
N = Set{Symbol}()
N = union(s, t, families, cars)
A = Set{Tuple}()
A = union(
    Set([(i,j) for i in s for j in families]), 
    Set([(i,j) for i in families for j in cars]),
    Set([(i,j) for i in cars, j in t]),
    Set([(i,j) for i in t for j in s])
)
# Important to note that the set A is a set of tuples, and we must into A appropriately in Julia.  This is a different way instead of doing NamedArray

# Put the data for the MCNF 
# Cost just has a '-1' on the (sink,source) arc (if minimizing)
c = Dict((i,j) => 0 for (i,j) in A)
c[:sink,:source] = -1.0

# Max flow.  All net supplies are 0
b = Dict(i => 0 for i in N)

# Upper bounds
u = Dict{Tuple, Float32}()
for i in s
    for j in families
        u[(i,j)] = 4
    end
end 
for i in families
    for j in cars
        u[(i,j)] = 2
    end
end 
u[(:car1,:sink)] = 4
u[(:car2,:sink)] = 3
u[(:car3,:sink)] = 3
u[(:car4,:sink)] = 4
u[(:sink,:source)] = Inf



In [ ]:
# Now we can just do MCNF max flow Model
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)
@variable(m, 0 <= x[a in A] <= u[a])
@objective(m, Min, sum(c[a]*x[a] for a in A))
@constraint(m, flow_balance[i in N], sum(x[(i,j)] for j in N if (i,j) in A) - sum(x[(j,i)] for j in N if (j,i) in A)== b[i])
#set_silent(m)
optimize!(m)

using Printf
@printf("Able to transport %.1f people\n", -objective_value(m))

